In [2]:
## Install glove-python module
!pip install -q glove-python

     |████████████████████████████████| 266kB 3.6MB/s 


In [3]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np
import itertools
from glove import Corpus, Glove
import pandas as pd
from keras import optimizers


Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def training_glove_weights(data):
    corpus = Corpus() 
    corpus.fit(data,window=10)
    glove = Glove(no_components=5, learning_rate=0.05)
    glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
    glove.add_dictionary(corpus.dictionary)
    glove.save('/content/drive/My Drive/extracted_data/mamidi_project/glove.model')

In [0]:
def getting_glove_weights():
    glove_model=Glove.load('/content/drive/My Drive/extracted_data/mamidi_project/glove.model')
    embedding_matrix2 = dict()
    embedding_matrix2 = glove_model.dictionary
    return embedding_matrix2

In [0]:
def getting_pretrained_glove_weights(vocabulary_size):
    embeddings_index = dict()
    f = open('/content/drive/My Drive/extracted_data/glove/glove.6B.100d.txt')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    embedding_matrix = np.zeros((vocabulary_size, 100))
    for word, index in tokenizer.word_index.items():
        if index > vocabulary_size - 1:
            break
        else:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
    return embedding_matrix

In [0]:

loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

NameError: ignored

In [8]:
df1 = pd.read_csv('/content/drive/My Drive/extracted_data/mamidi_project/data/without-1/cleanTrain.csv',dtype={'comment_text':str})
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 9 columns):
Unnamed: 0       159571 non-null int64
id               159571 non-null object
comment_text     159551 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(7), object(2)
memory usage: 11.0+ MB


In [9]:
comments = df1['comment_text'].values
comments.shape

(159571,)

In [10]:
l_comments = []
for i in comments:
  l_comments.append(str(i))
comments = np.asarray(l_comments)
comments.shape

(159571,)

In [0]:
labels = df1[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values

In [12]:
labels.shape[0]

159571

In [0]:
non_toxic_label = np.zeros(labels.shape[0])
for i in range(labels.shape[0]):
  flag = 0
  for j in range(labels.shape[1]):
    if(labels[i][j] == 1):
      flag = 1
      break
  if(flag == 0):
    non_toxic_label[i] = 1

In [0]:
nt_labels = np.hstack((labels, np.atleast_2d(non_toxic_label).T))

In [15]:
df2 = pd.read_csv('/content/drive/My Drive/extracted_data/mamidi_project/data/without-1/CleanTest.csv',dtype={'comment_text':str})
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63566 entries, 0 to 63565
Data columns (total 9 columns):
Unnamed: 0       63566 non-null int64
id               63566 non-null object
comment_text     63566 non-null object
toxic            63566 non-null int64
severe_toxic     63566 non-null int64
obscene          63566 non-null int64
threat           63566 non-null int64
insult           63566 non-null int64
identity_hate    63566 non-null int64
dtypes: int64(7), object(2)
memory usage: 4.4+ MB


In [0]:
test_comments = df2['comment_text'].values

In [0]:
test_labels = df2[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values

In [0]:
non_toxic_label = np.zeros(test_labels.shape[0])
for i in range(test_labels.shape[0]):
  flag = 0
  for j in range(test_labels.shape[1]):
    if(test_labels[i][j] == 1):
      flag = 1
      break
  if(flag == 0):
    non_toxic_label[i] = 1

In [0]:
nt_test_labels = np.hstack((test_labels, np.atleast_2d(non_toxic_label).T))

In [20]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(comments) + list(test_comments))
train = tokenizer.texts_to_sequences(comments)
test = tokenizer.texts_to_sequences(test_comments)
print('Average train sequence length: {}'.format(np.mean(list(map(len, train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, test)), dtype=int)))

Average train sequence length: 37
Average test sequence length: 35


In [21]:
max_sequence_len = max([len(x) for x in train])
print(max_sequence_len)
x_train = pad_sequences(train,maxlen = 100, padding = 'pre')
x_test = pad_sequences(test,maxlen = 100, padding = 'pre')


1384


In [0]:
total_words = len(tokenizer.word_index) + 1
print(total_words)

28710


In [0]:
embedding_matrix = getting_pretrained_glove_weights(20000)

In [23]:
model = Sequential()
input_len = max_sequence_len
model.add(Embedding(20000, 100 , weights=[embedding_matrix], input_length = 100))
# try 100 later
model.add(LSTM(100))
model.add(Dense(7,activation='sigmoid'))
opt_adam = optimizers.adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
        metrics=['accuracy'],optimizer=opt_adam)

In [25]:
hist = model.fit(x_train, nt_labels, batch_size=512, epochs=100)

Epoch 1/100
159571/159571 [==============================] - 348s 2ms/step - loss: 0.4134 - acc: 0.9579
Epoch 2/100
159571/159571 [==============================] - 312s 2ms/step - loss: 0.3839 - acc: 0.9624
Epoch 3/100
159571/159571 [==============================] - 312s 2ms/step - loss: 0.3641 - acc: 0.9627
Epoch 4/100
159571/159571 [==============================] - 311s 2ms/step - loss: 0.3499 - acc: 0.9587
Epoch 5/100
159571/159571 [==============================] - 313s 2ms/step - loss: 0.3378 - acc: 0.9573
Epoch 6/100
159571/159571 [==============================] - 311s 2ms/step - loss: 0.3266 - acc: 0.9579
Epoch 7/100
159571/159571 [==============================] - 311s 2ms/step - loss: 0.3169 - acc: 0.9587
Epoch 8/100
159571/159571 [==============================] - 318s 2ms/step - loss: 0.3067 - acc: 0.9585
Epoch 9/100
159571/159571 [==============================] - 338s 2ms/step - loss: 0.2981 - acc: 0.9599
Epoch 10/100
159571/159571 [==============================] - 33

In [0]:
def prediction():
    prediction = model.predict(x_test,verbose = 0)
    for i in range(2000):
        t_predict = prediction[i]
        for j in range(len(t_predict)):
            if(t_predict[j]>0.5):
                
                

SyntaxError: ignored

In [0]:
print(x_test.shape)

(2000, 100)


In [0]:
# for i in range(2000):
# print(x_test[0].shape)
# print(x_test[0])
# predict = model.predict(x_test)
print(predict)

[[7.7247369e-01 3.9043754e-02 3.9838210e-01 2.3434818e-02 5.5277014e-01
  4.2047799e-03]
 [7.3964947e-01 9.7271204e-03 5.7761294e-01 1.6030371e-03 1.8508497e-01
  5.6270063e-03]
 [6.8771958e-01 1.4411628e-02 5.0319332e-01 1.2647063e-02 2.5160670e-01
  9.9028051e-03]
 ...
 [8.0542046e-01 7.1884096e-03 2.7680922e-01 9.4074845e-02 1.3244742e-01
  1.6197562e-04]
 [8.6329925e-01 2.6610494e-04 1.9063950e-02 3.7279427e-03 9.2408359e-03
  8.5555434e-02]
 [8.0405450e-01 1.6441911e-02 5.0494462e-01 3.6408603e-02 1.4119947e-01
  1.2724996e-03]]


In [0]:
df = pd.read_csv('/content/drive/My Drive/extracted_data/mamidi_project/data/cleanTestLabels2000.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
Unnamed: 0       2000 non-null int64
id               2000 non-null object
toxic            2000 non-null int64
severe_toxic     2000 non-null int64
obscene          2000 non-null int64
threat           2000 non-null int64
insult           2000 non-null int64
identity_hate    2000 non-null int64
dtypes: int64(7), object(1)
memory usage: 125.1+ KB


In [97]:
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 707       
Total params: 2,081,107
Trainable params: 2,081,107
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.save("/content/drive/My Drive/extracted_data/mamidi_project/mamidi_lstm_f.h5")

In [0]:
predict = model.predict(x_test,verbose = 0)

In [0]:
print((test_labels))

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [0]:
print(predict)

[[4.58668113e-01 1.16527081e-05 4.92060184e-03 1.08480453e-05
  7.25272298e-03 1.45852566e-04]
 [5.87355018e-01 4.05609608e-05 2.79492140e-03 1.87814236e-04
  6.14144325e-01 1.22398138e-04]
 [7.08585143e-01 1.13248825e-05 2.79068947e-04 7.74860382e-05
  2.40087509e-04 1.08450651e-04]
 ...
 [7.93734312e-01 2.61157751e-04 2.83259153e-03 4.77850437e-04
  1.30653381e-04 1.09702349e-04]
 [1.44634157e-01 7.62939453e-06 7.18027353e-04 5.80221415e-04
  1.20672077e-01 7.11679459e-04]
 [5.76595485e-01 1.15334988e-05 3.60280275e-04 3.11166048e-04
  2.46629119e-03 1.31189823e-04]]


In [0]:
print(df['toxic'])

NameError: ignored

In [28]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

#calculate score
loss = log_loss(nt_test_labels,predict)
print("Log_loss : {}".format(loss))
predict = np.round(predict)
loss = hamming_loss(nt_test_labels,predict)
print("Hamming_loss : {}".format(loss*100))
accuracy = accuracy_score(nt_test_labels,predict)
print("Accuracy : {}".format(accuracy*100))

Log_loss : 1.9098137660721075
Hamming_loss : 4.80535416507477
Accuracy : 81.26986124657836
